## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Читаем  файл с данными и изучаем общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
#print(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

в таблице есть одинаковое количество пропущенных значений только в столбцах days_employed и total_income
все столбцы на первый взгляд в нужном типе данных

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#print(data['total_income'].isna().sum())
#print(data.isna().sum()) #- подсчёт пропущенных значений в каждом столбце
del data['days_employed']
for gender in data['gender'].unique():
    median = data.loc[data['gender'] == gender, 'total_income'].median()
    print(gender, median)
    data.loc[data['total_income'].isna() & (data['gender'] == gender), 'total_income'] = median
display(data.isna().sum())

F 134155.2834794494
M 167714.34371587454
XNA 203905.15726101527


children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Для заполнения пропусков в столбце с ежемесячным доходом группируем и ищем медиану по столбцам пола  и ежемесячного дохода
После заполнения пропусков в total_income проверочная информация выдала пропуски только в столбце со стажем. В вопросе задачи "влияет ли семейное положение и количество детей на факт погашения кредита в срок" информация со стажем минимально повлияет на результат и анализ => столбец days_employed по этим соображениям был удалён. Проверка таблицы после проделанного этапа.


Возможные причины отсутствия данных у одних и тех же людей в столбцах со стажем и ежемесячным доходом могут быть вызваны не предоставлением таких данных по собственному решению клиентов, невозможности документально подтвердить информацию, вероятны случаи отсутствия стажа как такового, как и ежемесячного дохода. Также, маловероятная, но имеющая место быть приxина техничего сбоя выгрузки файла. 

### Замена типа данных

In [3]:
display(data['children'].unique()) # просмотр уникальных значений в столбце с детьми, выявлены две аномалии: -1 ребёнок не может быть; 20 детей в семье возможны, но под сомнением
data['children'] = data['children'].apply(abs)
data.loc[data['children'] == 20, 'children'] = 2
display(data['children'].unique())
display(data['education'].unique()) # проверка уникальных значений в столбце с образованием, в столбце из-за разного регистра
data['education_lower'] = data['education'].str.lower()
display(data.head())
display(data['education_lower'].unique())
del data['education']
display(data.head())

data['total_income'] = data['total_income'].astype('int')
data.info()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([1, 0, 3, 2, 4, 5])

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower
0,1,42,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее
1,1,36,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее
2,0,33,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее
3,3,32,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее
4,0,53,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
education_lower     21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод

Замена отрицательных значений в столбце количества детей, так как значения "-1" не может быть.
Замена количества детей в размере 20 штук, так как по логике это аномальное явление.
Типы данных соответствуют столбцам => замены не требуется.

Проверка уникальных значений в столбце с образованием, в столбце из-за разного регистра формируется больше уникальных значений. 
Приведение всех значений к нижнему регистру и вывод в новом столбце education_lower и проверка на уникальность этого столбца. Удаление изначального столбца education после отработки

Доработка после комментариев:
Приведение столбца total_income к целочисленному типу данных

### Обработка дубликатов

In [4]:
print ('Дубликатов в таблице:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print ('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 71
Дубликатов в таблице: 0


### Вывод

Выявлено 71 повторение и удалено методом .drop_duplicates() с изменением индексов и удалением старых.
Предполагаемая причина появления дубликатов - техническая при выгрузке файла.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem() 
text_unique = data['purpose'].unique()
text = ' '.join(text_unique)
lemmas = m.lemmatize(text)
#print(lemmas)
from collections import Counter
print(Counter(lemmas))
data['lemmas_purpose'] = data['purpose'].apply(m.lemmatize)
#print(data.head())

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Вывод

В столбце с назначением кредита выводим уникальные значения и формируем их в одну строку. После чего применяем лематизацию и подсчитываем количество лемм в данном столбце. Далее формируем новый столбец с лемматизацией значений в столбце purpose. 

### Категоризация данных

In [6]:
def category(lemmas_purpose):
    if 'недвижимость' in lemmas_purpose or 'жилье' in lemmas_purpose:
        return 'недвижимость'
    if 'автомобиль' in lemmas_purpose:
        return 'автомобиль'
    if 'образование' in lemmas_purpose:
        return 'образование'
    if 'операция' in lemmas_purpose:
        return 'операция'
    if 'свадьба' in lemmas_purpose:
        return 'свадьба'
    if 'строительство' in lemmas_purpose:
        return 'строительство'
    if 'ремонт' in lemmas_purpose:
        return 'ремонт'
    return 'ошибка'
data['purpose_final'] = data['lemmas_purpose'].apply(category)
print(data.head())
print(data['purpose_final'].value_counts())

   children  dob_years  education_id     family_status  family_status_id  \
0         1         42             0   женат / замужем                 0   
1         1         36             1   женат / замужем                 0   
2         0         33             1   женат / замужем                 0   
3         3         32             1   женат / замужем                 0   
4         0         53             1  гражданский брак                 1   

  gender income_type  debt  total_income                     purpose  \
0      F   сотрудник     0        253875               покупка жилья   
1      F   сотрудник     0        112080     приобретение автомобиля   
2      M   сотрудник     0        145885               покупка жилья   
3      M   сотрудник     0        267628  дополнительное образование   
4      F   пенсионер     0        158616             сыграть свадьбу   

  education_lower                        lemmas_purpose purpose_final  
0          высшее               [покуп

### Вывод

Создаётся новый столбец с категоризацией лемм из столбца lemmas_purpose и подсчёт количества назначений кредита каждой категории. Самый частый кредит - ипотека. Второй приоритет - автомобиль. 

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#print(data['total_income'].mean())
data_pivot_children = data.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count').fillna(0)
data_pivot_children.style.format({'total_income':'{:.0f}'})
data_pivot_children['no_debt'] = data_pivot_children[0] / (data_pivot_children[0] + data_pivot_children[1]) * 100
data_pivot_children['debt'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1]) * 100
print(data_pivot_children)

debt            0       1     no_debt      debt
children                                       
0         13028.0  1063.0   92.456178  7.543822
1          4410.0   445.0   90.834192  9.165808
2          1926.0   202.0   90.507519  9.492481
3           303.0    27.0   91.818182  8.181818
4            37.0     4.0   90.243902  9.756098
5             9.0     0.0  100.000000  0.000000


### Вывод

Для ответа на вопрос о зависимости наличия детей и возвратом кредита в срок была создана сводная таблица с индексом количества детей в семье и столбцами факта возврата или просроченного кредита. Для наглядности были добавлены столбцы соотношения невозвращенных кредитов к полному количеству кредитов по одной категории наличия детей. В результате мы можем наблюдать такие выводы:
*  самые ответственные люди статистически с пятью и более детьми, вводных данных с семьями с большим количеством детей не очень много, но некоторые выводы относительно ответственности можно сделать, взяв хотя бы факт отсутствия задолженности на всё количество кредитов 
* далее статистика показывает наиболее возвращаемые кредиты у людей с тремя детьми
* бездетных людей больше - больше и точнее статистика, они на третьем месте по надёжности возврата кредита
* на одном уровне надёжности находятся люди с одним или двумя детьми
* наименее ответственные люди перед долговыми обязательствами по статистике - с четырьмя детьми

С одной стороны, количество детей говорит о стабильности потенциального заемщика, но надо не забывать, что банки обязательно будут вычитать минимальное содержание каждого ребенка из дохода. Люди, заводящие детей, принимающие ответственность не только за себя, но и за другую жизнь, более осознаны в принятии решения вступления в долговые обязательства. Дети являются некими "якорями", привязывают к месту (детские сады, школы, их социальная жизнь), семье уже труднее сменить место жительства и скрываться от кредиторов. Но при этом наличие детец сильно увеличивает ежемесячные статьи расходов семьи, что может помешать выплачивать всю ежемесячную сумму погашения кредита или же вообще пропускать выплаты.  

In [8]:
data.groupby('children').agg({'debt':['count', 'sum', 'mean']})

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4855   445  0.091658
2          2128   202  0.094925
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
data_pivot_family = data.pivot_table(index='family_status', columns='debt', values='total_income', aggfunc='count').fillna(0)
data_pivot_family.style.format({'total_income':'{:.0f}'})
data_pivot_family['no_debt'] = data_pivot_family[0] / (data_pivot_family[0] + data_pivot_family[1]) * 100
data_pivot_family['debt'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1]) * 100
print(data_pivot_family)

debt                       0    1    no_debt      debt
family_status                                         
Не женат / не замужем   2536  274  90.249110  9.750890
в разводе               1110   85  92.887029  7.112971
вдовец / вдова           896   63  93.430657  6.569343
гражданский брак        3763  388  90.652855  9.347145
женат / замужем        11408  931  92.454818  7.545182


### Вывод

Точно таким же способом собирается сводная таблица с индексом по семейному статусу и столбцами по наличию задолженности с добавлением столбцом соотношений невозвращенных кредитов к полному количеству кредитов по одной категории семейного статуса. Из этой таблицы можно сделать выводы:
* самый высокий процент отдаваемости кредита в срок у вдовствующих людей
* вторым, вызывающим доверие для выдачи кредита, являются люди в разводе
* как ни удивительно, но женатые люди занимают третье место в надёжности кредитных обязательств
* гражданский брак на четвертом месте 
* самые безответственные люди являются неженатыми по данным сводной таблицы

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
def total_income_group(row):
    total_income_group = row
    if total_income_group < 50000:
        return 'низкий доход'
    if 50000 < total_income_group < 100000:
        return 'средний доход'
    if total_income_group > 100000:
        return 'высокий доход'
data['total_income_group'] = data['total_income'].apply(total_income_group)
#data.info()
data_pivot_income = data.pivot_table(index='total_income_group', columns='debt', values='total_income', aggfunc='count').fillna(0)
data_pivot_income.style.format({'total_income':'{:.0f}'})
data_pivot_income['no_debt'] = data_pivot_income[0] / (data_pivot_income[0] + data_pivot_income[1]) * 100
data_pivot_income['debt'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1]) * 100
print(data_pivot_income)

debt                    0     1    no_debt      debt
total_income_group                                  
высокий доход       15604  1387  91.836855  8.163145
низкий доход          349    23  93.817204  6.182796
средний доход        3760   331  91.909069  8.090931


### Вывод

Для выявления надёжности заёмщиков по уровням заработных плат было сгруппировано три категории на основании данных из Росстата за 2020 год по средним зарплатам в России: низкий, средний и высокий доходы. Для проведения детального анализа воспользовалась тем же методом сводных таблиц, что и в двух предыдущих случаях. По итогу:
* люди с низкими зарплатами самые ответственные 
* среднее значение надёжности занимают люди со средней зарплатой
* самые ненадёжные по статистике данного исследования - люди с высокой зарплатой

#спасибо, я изучу побольше про квантилей и в следующей работе с похожими задачами использую, на данный момент я уже не успею в срок изучить и применить этот метод:)

In [11]:
data['total_income'].quantile([0.25, 0.5, 0.75])

0.25    107623.00
0.50    141943.00
0.75    195813.25
Name: total_income, dtype: float64

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
data_pivot_purpose = data.pivot_table(index='purpose_final', columns='debt', values='total_income', aggfunc='count').fillna(0)
data_pivot_purpose.style.format({'total_income':'{:.0f}'})
data_pivot_purpose['no_debt'] = data_pivot_purpose[0] / (data_pivot_purpose[0] + data_pivot_purpose[1]) * 100
data_pivot_purpose['debt'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1]) * 100
print(data_pivot_purpose)

debt               0    1    no_debt      debt
purpose_final                                 
автомобиль      3903  403  90.640966  9.359034
недвижимость   10029  782  92.766627  7.233373
образование     3643  370  90.779965  9.220035
свадьба         2138  186  91.996558  8.003442


### Вывод

Самый возвращаемый кредитный продукт: ипотека. 
Ипотеку берут осознанно и подготовившись, составив планы и цели на ближайшие 20 лет, на кону половина жизни человека, поэтому ипотека самый надёжный банковский продукт по статистике. 
Ипотеку берут семьи с детьми или готовящиеся к рождению детей, для улучшения жилищных условий и предоставления материальных благ в будущем своим потомкам. Для среднестатистического человека стоимость квартиры является достаточно великой, весомой и устрашающей, чтобы попадать в ситуации лишения квартиры, имущества или причинение проблем для своей семьи за неуплату долгов.

Второй по возвращаемости кредит: свадьба.
Свадьба является радостным, ярким, жизнерадостным и светлым событием для каждой пары, поэтому для проведения торжества опрометчиво берут кредит, подходя к обязательствам не так осознанно, возможно, думая, что подарки на свадьбу окупят занимаемые средства. Но на свадьбу в среднем берут не крупную сумму, что позволяет за небольшой срок её выплатить.

Третье место делят кредиты на автомобиль и образование.
Автомобиль является признаком достатка. Также приобретается семьями для упрощения смены локаций и перемешения комфортно. Автомобиль требует массы расходов, такие как топливо, налог, штрафы, уход, ремонт, страхование. Автомобиль - движимое имущество, теряющее свою ликвидность ежедневно. Выдавая кредит на машину настоятельно советую смотреть на стаж вождения для определения высоты рисков попадания в аварию. Свежеиспечённый водитель, попав в аварию, повредив траспортное средство, потяеряет некоторую долю мотивации выплачивать кредит за эту машину, которой не может пользоваться.
Образование является благородным делом почти для каждого человека в мире. С точки зрения людей оно поможет преодолеть преграду на пути к более финансово комфортной жизни. Чаще всего люди не сопоставляют ценность приобретённых знаний и конвертацию их в материальные блага и попадают в ситуации финансового неблагополучия, где возврат кредита не является первостепенной задачей.

### Шаг 4. Общий вывод

В проекте поднимались вопросы о зависимости наличия детей и семейного положения на возвращаемость кредитов в срок. В датафрейме, предоставленном дл исследования, были проведены первоначальные шаги для качественного анализа данных:
* выгрузка файла csv и предварительный просмотр ифнормации о датафрейме
  на первый взгляд таблица была подготвлена с соответствующими типами данных
  
* выявление пропусков
  в датафрейме было всего два столбца с пропусками, где один был ликвидирован после логического вывода о минимальном влиянии содержащейся информации в столбце с пропущенными значениями стажа клиентов и второй столбец с ежемесячной зарплатой клиентов(критически важная информация дл исследования)  был обработан на заполнение пропусков медианой - средним значением всего столбца
  предполагаемые причины появления пропусков могло являться отсутствие внесения этой информации в базу по решению клиента 
  
* удаление дубликатов и правка данных
 в таблице был найден 71 дубликат, по соображениям потери малого количества иформации в процентном соотношении (0.00003531) дубликаты были удалены 
 в столбце с детьми было выявлено две привлекающие внимание ошибки: отрицательное количество детей и аномально большое количество детей в семье (20 штук) в 76 строках. При детальном рассмотрении было определено, что это явно ошибка, так как 21-летний человек не может иметь 20 детей. Отрицательное количесто стало положительным, количество 20 детей исправлено на 2.
 причины дубликатов могли стать сбои в выгрузке файла, причины в количестве детей же является ошибкой человеческого фактора с вероятностью в 80%
 
* лемматизация столбца с назначением кредита для категоризации кредита 

В результате исследования можно сделать вывод, что самый надёжный заёмщик: вдовец с тремя или пятью детьми с низкой зарплатой с целью приобрести жильё. 
 